<a href="https://colab.research.google.com/github/besherh/Machine-Learning-Course/blob/master/MLOPS/LR__MLflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLflow + Colab – Example project
This project shows how you can easily log experiments with Google Colab, directly to an MLflow remote.

It uses DAGsHub MLflow remote server, which is a free hosted MLflow remote.

Instructions:
1. [Sign up to DAGsHub](https://dagshub.com/user/sign_up)
1. [Create an access token](https://dagshub.com/user/settings/tokens)
1. [Create a new project](https://dagshub.com/repo/create)
1. [Open the notebook in Google Colab](https://dagshub.com/Dean/mlflow-colab-example/src/master/mlflow-colab.ipynb)
1. Run the cell
1. Fill in your DAGsHub username, access token and project name.
1. Run the visualization cell if you want to see your experiments inside Colab.

In [5]:
!pip install mlflow --quiet

import mlflow
import os
from getpass import getpass

os.environ['MLFLOW_TRACKING_USERNAME'] = input()
os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('')
os.environ['MLFLOW_TRACKING_PROJECTNAME'] = input('')

mlflow.set_tracking_uri(f'https://dagshub.com/' + os.environ['MLFLOW_TRACKING_USERNAME'] 
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + '.mlflow')

with mlflow.start_run(run_name="MLflow on Colab"):
  mlflow.log_metric("m1", 2.0)
  mlflow.log_param("p1", "mlflow-colab")

     |████████████████████████████████| 16.5 MB 5.3 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 182 kB 56.9 MB/s 
     |████████████████████████████████| 209 kB 55.1 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 575 kB 63.5 MB/s 
     |████████████████████████████████| 147 kB 77.4 MB/s 
     |████████████████████████████████| 78 kB 7.5 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 140 kB 75.4 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
besher.alhalabi
··········
mlops_demo


In [6]:
# See your experiments table inside Colab!
import IPython
display(IPython.display.IFrame("https://dagshub.com/"+ os.environ['MLFLOW_TRACKING_USERNAME'] 
                        + '/' + os.environ['MLFLOW_TRACKING_PROJECTNAME'] + "/experiments/#/",'100%',600))

In [7]:
!pip list | grep mlflow

mlflow                        2.0.1


In [8]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn




In [9]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


In [10]:
csv_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(csv_url, sep=";")


In [11]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

 # The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [17]:
alpha = 0.1
l1_ratio = 0.1


In [18]:
 with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.697078353435743
  MAE: 0.5446639367673861
  R2: 0.31170617822346913


Successfully registered model ''.
2022/12/07 19:16:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetWineModel, version 3
Created version '3' of model 'ElasticnetWineModel'.
